In [3]:
#from library_dicom.dicom_processor.tools.pyradiomic import *
import SimpleITK as sitk 
import os 
import numpy as np 
import csv
import json 
from library_dicom.dicom_processor.model.csv_reader.CsvReader import CsvReader

In [2]:
csv_path= '/media/deeplearning/LACIE SHARE/PVAB/NIFTI/no_suv/PVAB_NIFTI_no_suv.csv'
with open(csv_path, 'r') as csv_file :
    reader = csv.reader(csv_file, delimiter = ',') #liste pour chaque ligne 
    csv_file_ = []
    for row in reader :
        csv_file_.append(row)
        
del csv_file_[0] #enlever première ligne

In [3]:
print("Nombre de mask à traiter : ", len(csv_file_))

Nombre de mask à traiter :  5


#### - Keep only pet 0 

In [4]:
pet0 = []
for row in csv_file_ : 
    if 'PET0' in row : 
        pet0.append(row)
print(len(pet0))

5


In [5]:
#get date 
json_path = '/media/deeplearning/LACIE SHARE/PVAB/pvab_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

#print(len(dataset))
for row in pet0 : 
    patient_id = row[0]
    for liste in dataset : 
        if patient_id in liste : 
            row.append(liste[-1])

#check if every study has its csv 
for row in pet0 : 
    if len(row) != 9 : 
        print(row)


In [6]:
#change path if necessary and get date
for row in pet0 : 
    #change csv_path 
    csv_file_path = row[-1]
    x = csv_file_path.replace('oncopole', 'deeplearning')
    row[-1] = x 

    #change pet and mask path 
    pet = row[5]
    mask= row[6]
    x = pet.replace('oncopole', 'deeplearning')
    y = mask.replace('oncopole', 'deeplearning')
    row[5] = x 
    row[6] = y 

    #get date 

    reader = CsvReader(row[-1])
    date = reader.get_date()
    row.append(date)


In [7]:
for row in pet0 : 
    if len(row) != 10 : 
        print(row)

In [9]:
pet0[0]

['13014101443001',
 '13014101443001',
 '1.2.276.0.7230010.3.1.2.447480576.1.1612345812.227572',
 'PET0',
 '',
 '/media/deeplearning/LACIE SHARE/PVAB/NIFTI/no_suv/13014101443001/PET0/1.2.276.0.7230010.3.1.2.447480576.1.1612345812.227572_nifti_PT.nii',
 '/media/deeplearning/LACIE SHARE/PVAB/NIFTI/no_suv/13014101443001/PET0/1.2.276.0.7230010.3.1.2.447480576.1.1612345812.227572_nifti_mask.nii',
 '/media/deeplearning/LACIE SHARE/PVAB/PVAB_csv/883/13014101443001_PET0_41.csv',
 ' Sep 30_2016']

In [10]:
for nifti in pet0: 
    try : 
        print(pet0.index(nifti))
        mask_path = nifti[6]
        pet_path = nifti[5]
        liste_center = get_center_of_mass(mask_path, thresh = True, pet_path = pet_path)
        d_max = calcul_distance_max(liste_center)
        nifti.append(d_max)
    except Exception as err : 
        print(nifti)
        print(err)

0
(169, 169, 234, 26)
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Ave

In [11]:
nifti_directory = '/media/deeplearning/LACIE SHARE/PVAB'
filename = 'PVAB_dMax_v3.csv'


with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT ID", "DATE", "STUDY_UID", "D_MAX"])
    for row in pet0: 
        csv_writer.writerow([row[1], row[-2], row[2], row[-1]])
